In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool
%matplotlib notebook
%run utils.ipynb
%run all_create.ipynb

[32.31738261840044, 72.15091393575189, 116.0946312221527, 172.07253929769357, 260.3569293324722, 360.3101542051449, 500.2151479081191, 660.12508634391, 820.1546870692688, 1020.1106909393304]


In [ ]:
"""
def predict_next_hit(hits_so_far):
    n_steps = len(hits_so_far[0])
    n_input_features = 5
    n_neurons = 200
    n_outputs = 3

    X = tf.placeholder(tf.float32, [None, n_steps, n_input_features], name='input')
    
    lstm = tf.contrib.rnn.LSTMCell(num_units = n_neurons, use_peepholes = True)
    lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = n_outputs, reuse=tf.AUTO_REUSE)
    rnn_outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
    print_op = tf.Print(rnn_outputs, [rnn_outputs], name='print_pred')
    init = tf.global_variables_initializer()
    
    with tf.Session() as lstm3sess:

        init.run()
        initial_step = global_step.eval()
        saver = tf.train.Saver()
        writer = tf.summary.FileWriter('./logs/lstm3', lstm3sess.graph)
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/lstm3/'))
        # if that checkpoint exists, restore from checkpoint
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(lstm3sess, ckpt.model_checkpoint_path)

        x_data = hits_so_far
        test_print = lstm3sess.run(print_op, feed_dict={X:x_data})
        return [test_print[0][-1][0], test_print[0][-1][1], test_print[0][-1][2]]
"""

In [2]:
def reconstruct_track_from_seed (feed_seed, hits_array):
    mse_hitid = []
    for j in range(7):
        predicted_hit = prediction.pred(feed_seed)[j+2]
        predicted_hit_xyz = polar_to_cartesian(predicted_hit[0], predicted_hit[1], predicted_hit[2])
        (real_hit_id, dist_squared) = calc_dist(predicted_hit_xyz[0], predicted_hit_xyz[1], predicted_hit_xyz[2], hits_array)
        hit_index = int (np.round (real_hit_id - 1))
        assert (hits_array[hit_index,0] == real_hit_id)
        x, y, z, l, v = hits_array[hit_index,1], hits_array[hit_index,2], hits_array[hit_index,3], hits_array[hit_index,5], hits_array[hit_index,4]
        r, phi, z = cartesian_to_3d_polar (x, y, z)
        to_append = np.array ([[r, phi, z, l, v]])
        feed_seed = np.append (feed_seed, to_append, axis = 0)
        mse_hitid.append ([dist_squared, real_hit_id])
        continue
    return (feed_seed, mse_hitid)

In [3]:
def reconstruct_event (seeds, hits):
    hits_array = np.array (hits.get_values())
    hits_from_seeds = rpzlv_from_seeds (seeds, hits)
    seeds_generated = next_seed(hits_from_seeds)
    hits_from_tracks = []
    mse_hitid = []
    count = 0
    for feed_seed in seeds_generated:
        feed_seed = np.array (feed_seed[0])
        seed_mse_hitid = [[0, seeds[count,0]],[0, seeds[count,1]],[0, seeds[count,2]]]
        (feed_seed, track_mse_hitid) = reconstruct_track_from_seed (feed_seed, hits_array)
        hits_from_tracks.append (feed_seed)
        mse_hitid.append (seed_mse_hitid + track_mse_hitid)
        count += 1
       
    return (hits_from_tracks, mse_hitid)



In [4]:
def reconstruct_test_events ():
    itr = load_dataset ('./data/test_data', parts = ['hits'])
    for event_data in itr:
        event_id = event_data[0]
        print ('now working on event number ' + str (event_id))
        if (event_id <= 43):
            continue
        hits = event_data[1]
        hits_trans = trans_to_cylindrical (hits)
        seeds = np.load ('./seeds/test_event_' + str(event_id) + '_.npy')
        yield reconstruct_event (seeds, hits), event_id#yield (hits_from_event, hit_ids)

In [5]:
prediction = predict_engine()
prediction.load()
hit_df, _, _,_ = load_data_single_event(1050)
start_time = time.time()
seeds = np.load('./seeds/training_event_v2_1050.npy')
reconstructed_tracks, mse_hitid = reconstruct_event(seeds, hit_df)
np.save ('./experimental/hits_from_tracks_event_' + str (1050) + '.npy', np.array (hits_from_tracks))
np.save ('./experimental/mse_hitid_event_' + str (1050) + '.npy', np.array (mse_hitid))


INFO:tensorflow:Restoring parameters from ./checkpoints/allv1/model.ckpt


NameError: name 'event_id' is not defined

In [8]:
np.save ('./experimental/hits_from_tracks_event_' + str (1050) + '.npy', np.array (reconstructed_tracks))
np.save ('./experimental/mse_hitid_event_' + str (1050) + '.npy', np.array (mse_hitid))


In [ ]:
prediction = predict_engine()
prediction.load()
start_time = time.time()
for (hits_from_tracks, mse_hitid), event_id in reconstruct_test_events():
    print ('now saving event number ' + str (event_id))
    print ('elapsed time is ' + str ((time.time() - start_time)/60/60) + ' hours.')
    np.save ('./reconstructed_test_events/hits_from_tracks_event_' + str (event_id) + '.npy', np.array (hits_from_tracks))
    np.save ('./reconstructed_test_events/mse_hitid_event_' + str (event_id) + '.npy', np.array (mse_hitid))


In [ ]:
#np.save ('./tracksToPlot.npy', np.array (hits_from_tracks))

In [ ]:
mapping = np.load('./mse_hitid.npy')
track_cand = np.load('./hits_from_tracks.npy')

In [ ]:

mapping.shape

In [ ]:
track_cand.shape

In [ ]:
seeds.shape